# Programming Hidden Markov Models (60 P)

In this exercise, you will experiment with hidden Markov models, in particular, applying them to modeling character sequences, and analyzing the learned solution. As a starting point, you are provided in the file `hmm.py` with a basic implementation of an HMM and of the Baum-Welch training algorithm. The names of variables used in the code and the references to equations are taken from the HMM paper by Rabiner et al. downloable from ISIS. In addition to the variables described in this paper, we use two additional variables: $Z$ for the emission probabilities of observations $O$, and $\psi$ (i.e. psi) for collecting the statistics of Equation (40c).


##Question 1: Analysis of a small HMM (30 P)

We first look at a toy example of an HMM trained on a binary sequence. The training procedure below consists of 100 iterations of the Baum-Welch procedure. It runs the HMM learning algorithm for some toy binary data and prints the parameters learned by the HMM (i.e. matrices $A$ and $B$).

###Question 1a: Qualitative Analysis (15 P)

* *Run* the code several times to check that the behavior is consistent.
* *Describe* qualitatively the solution $A,B$ learned by the model.
* *Explain* how the solution $\lambda = (A,B)$ relates to the sequence of observations $O$ that has been modeled.


In [1]:
import numpy,hmm

O = numpy.array([1,0,1,0,1,1,0,0,1,0,0,0,1,1,1,0,1,0,0,0,1,1,0,1,1,0,0,1,1,
                 0,0,0,1,0,0,0,1,1,0,0,1,0,0,1,1,0,0,0,1,0,1,0,1,0,0,0,1,0,
                 0,0,1,0,1,0,1,0,0,0,1,1,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,
                 0,1,1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,1,1,
                 1,0,0,0,1,1,0,0,1,0,1,1,1,0,0,1,1,0,0,0,1,1,0,0,1,1,0,0,1,
                 0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,
                 0,0,1,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,0,0,0,1,1,0,0])

for i in range(10):
    print 'Iteration nr: {}'.format(i)
    hmmtoy = hmm.HMM(4,2)

    for k in range(100):
        hmmtoy.loaddata(O)
        hmmtoy.forward()
        hmmtoy.backward()
        hmmtoy.learn()

    print('A')
    print("\n".join([" ".join(['%.3f'%a for a in aa]) for aa in hmmtoy.A]))
    print(' ')
    print('B')
    print("\n".join([" ".join(['%.3f'%b for b in bb]) for bb in hmmtoy.B]))
    print(' ')
    print('Pi')
    print("\n".join(['%.3f'%b for b in hmmtoy.Pi]))
    print '\n\n'

Iteration nr: 0
A
0.000 1.000 0.000 0.000
0.000 0.000 1.000 0.000
0.000 0.000 0.000 1.000
1.000 0.000 0.000 0.000
 
B
0.000 1.000
0.720 0.280
0.800 0.200
0.880 0.120
 
Pi
1.000
0.000
0.000
0.000



Iteration nr: 1
A
0.000 0.000 0.000 1.000
0.000 0.000 1.000 0.000
1.000 0.000 0.000 0.000
0.000 1.000 0.000 0.000
 
B
0.880 0.120
0.720 0.280
0.800 0.200
0.000 1.000
 
Pi
0.000
0.000
0.000
1.000



Iteration nr: 2
A
0.000 1.000 0.000 0.000
0.000 0.000 0.000 1.000
1.000 0.000 0.000 0.000
0.000 0.000 1.000 0.000
 
B
0.800 0.200
0.880 0.120
0.720 0.280
0.000 1.000
 
Pi
0.000
0.000
0.000
1.000



Iteration nr: 3
A
0.000 1.000 0.000 0.000
0.000 0.000 1.000 0.000
0.000 0.000 0.000 1.000
1.000 0.000 0.000 0.000
 
B
0.880 0.120
0.000 1.000
0.720 0.280
0.800 0.200
 
Pi
0.000
1.000
0.000
0.000



Iteration nr: 4
A
0.000 1.000 0.000 0.000
0.000 0.000 1.000 0.000
0.000 0.000 0.000 1.000
1.000 0.000 0.000 0.000
 
B
0.880 0.120
0.000 1.000
0.720 0.280
0.800 0.200
 
Pi
0.000
1.000
0.000
0.000



Iteration 

**ANSWER:**

The beheavior is not consistent.

In several iterations, the model learned a parameter $A$ that ensures the transition to a state different from the actual, i.e., $\forall i, \exists j \neq i \mid a_{ij} = 1$

On the other hand, the parameter $B$, with some exceptions, was more consistently learned, changing generally only the row order between iterations. This parameter shows that the hidden states have a strong "preference" for either the symbol `1` or the symbol `0`, i.e., $\forall i, b_{i0} > 0.6 \vee b_{i1} > 0.6 $.

Given that the solution is not stable, it's difficult to say what the parameters $(A, B)$ are explaining about $O$, beyond what was already said above.

###Question 1b: Finding the best number $N$ of hidden states (15 P)

For the same sequence of observations as in Question 1a, we would like to determine automatically what is a good number of hidden states $N = \mathrm{card}(S)$ for the model.

* *Split* the sequence of observations into a training and test set (you can assume stationarity).
* *Train* the model on the training set for several iteration (e.g. 100 iterations) and for multiple parameter $N$.
* *Show* for each choice of parameter $N$ the log-probability $\log p(O | \lambda)$ for the test set. (If the results are unstable, perform several trials of the same experiment for each parameter $N$.)
* *Explain* in the light of this experiment what is the best parameter $N$.

In [3]:
forward(markov_guy.A, markov_guy.B, markov_guy.Pi, O)

NameError: name 'forward' is not defined

In [4]:
def forward(A, B, Pi, O):
    # Initialization
    Z = numpy.array([B[:, O[t]] for t in range(len(O))])
    alpha = numpy.empty([len(O), len(Pi)])
    alpha[:] = numpy.NaN
    alpha[0] = Pi * Z[0]
        
    # Induction
    for t in range(len(O)-1):
        alpha[t + 1] = numpy.dot(alpha[t], A) * Z[t + 1]
        
    # Termination
    return alpha[-1].sum()


L = O.shape[0]
o_idx = numpy.arange(L)
train_size = int(L * 0.8)
folds = []
folds_nr = 5
# Generate cross validation folds
for i in range(folds_nr):
    train_idx = numpy.random.choice(o_idx, train_size, replace=False)
    test_idx = numpy.array(list(set(o_idx) - set(train_idx)))
    train = O[train_idx]
    test = O[test_idx]
    folds.append((train, test))


epochs = 100
result = numpy.zeros((folds_nr, epochs))
for i, fold in enumerate(folds):
    print "Training on fold nr: {}".format(i)
    train, test = fold
    for n in range(1, epochs + 1):
        markov_guy = hmm.HMM(n, 2)
        for k in range(100):
            markov_guy.loaddata(train)
            markov_guy.forward()
            markov_guy.backward()
            markov_guy.learn()
        result[i, n - 1] = forward(markov_guy.A, markov_guy.B, markov_guy.Pi,test)

result

Training on fold nr: 0
Training on fold nr: 1
Training on fold nr: 2
Training on fold nr: 3
Training on fold nr: 4


array([[  1.31746754e-012,   3.33759923e-012,   2.04453146e-012,
          8.21741692e-013,   2.70525378e-013,   7.28534266e-018,
          2.29129546e-021,   3.27797277e-017,   1.13041722e-023,
          1.77933904e-016,   7.47192218e-027,   2.40791494e-023,
          1.17996034e-023,   7.00220028e-016,   2.47091805e-019,
          5.10197198e-019,   3.93941933e-018,   9.73107143e-034,
          2.11850377e-017,   6.76612151e-028,   1.93905187e-028,
          3.10540232e-022,   3.98114848e-022,   9.28864640e-031,
          7.99312170e-026,   6.78642545e-019,   1.47474508e-023,
          4.78726516e-026,   4.23479444e-022,   3.04449246e-024,
          8.88207606e-029,   5.73065677e-031,   1.15618548e-023,
          3.56621579e-032,   1.16765836e-058,   6.25535091e-020,
          1.42559086e-021,   3.20053917e-017,   4.64324339e-024,
          1.57365904e-025,   3.39345695e-033,   9.40528639e-044,
          1.04002720e-051,   2.05374339e-027,   1.27290540e-028,
          2.49298989e-020

In [5]:
log_result = numpy.log(result)
log_result

C:\Program Files\Anaconda\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


array([[ -27.35530976,  -26.42576936,  -26.91585247,  -27.82735029,
         -28.93841048,  -39.4606672 ,  -47.52516959,  -37.95672141,
         -52.83687035,  -36.26511952,  -60.15864522,  -52.08069593,
         -52.79397631,  -34.89513706,  -42.84452701,  -42.11948964,
         -40.07549834,  -76.01256916,  -38.39323651,  -62.56045458,
         -63.81018348,  -49.52372877,  -49.2753017 ,  -69.15134505,
         -57.78863103,  -41.83419241,  -52.57097199,  -58.30125312,
         -49.21353726,  -54.14870803,  -64.59093238,  -69.63430774,
         -52.81433094,  -72.41121795, -133.39493505,  -44.21826462,
         -47.99970059,  -37.98062729,  -53.7266291 ,  -57.11122382,
         -74.76345892,  -99.07247218, -117.39259287,  -61.45013334,
         -64.2310806 ,  -45.13821911,  -44.395211  ,  -54.37511144,
         -71.23290603,  -55.58998333,  -59.20394249,  -41.1163897 ,
        -142.40593984,  -49.6374127 ,  -57.03728305,  -44.7842972 ,
         -56.64207477,  -62.95167094,  -73.67197

In [6]:
median_logp = numpy.median(log_result, axis=0)
print(median_logp)
median_p = numpy.median(result, axis=0)
max_logp = numpy.argmax(median_logp)
print("Best N: {}."
      "\nCorresponding median (over CV folds) logp(O): {}"
      "\nCorresponding median (over CV folds) p(O): {}".format(
            max_logp + 1, median_logp[max_logp], median_p[max_logp]
      )
)

[ -26.54422127  -26.93104562  -27.56070251  -28.46876611  -30.93859897
  -39.4606672   -47.52516959  -37.95672141  -52.83687035  -36.69020975
  -60.15864522  -52.08069593  -52.79397631  -50.43355215  -43.43464859
  -45.32499521  -43.40922184  -76.01256916  -43.5865317   -69.19422924
  -65.04998736  -96.63840571  -78.50551694  -69.15134505  -57.78863103
  -44.56628385  -68.85232286  -79.91359023  -58.57592418  -78.74911579
  -74.82539203  -69.63430774  -72.31979526 -105.35626767 -133.39493505
  -85.27500546  -73.79841359  -65.53479538  -79.98078864  -75.13665086
 -106.7089515  -107.50992223 -117.39259287 -127.0495115  -100.94512402
  -87.0280037   -95.10056512 -104.91188684 -165.29011334  -73.37371367
  -96.91329359  -83.82448375 -142.40593984 -120.13313492  -99.47760484
  -70.98596988 -114.21655314  -92.11189424  -92.44007739  -76.29807185
 -105.08064759 -105.54082061 -102.30054928 -185.51316589 -119.58805003
 -131.76006149 -103.57355358 -109.68068064 -115.83359781 -114.96039367
 -109.

## Question 2: Text modeling and generation (30 P)

We would like to train an HMM on character sequences taken from English text. We use the 20 newsgroups dataset that is accessible via scikits-learn http://scikit-learn.org/stable/datasets/twenty_newsgroups.html. (For this, you need to install scikits-learn if not done already.) Documentation is available on the website. The code below allows you to (1) read the dataset, (2) sample HMM-readable sequences from it, and (3) convert them back into string of characters.

In [56]:
from sklearn.datasets import fetch_20newsgroups

# Download a subset of the newsgroup dataset
newsgroups_train = fetch_20newsgroups(subset='train',categories=['sci.med'])
newsgroups_test  = fetch_20newsgroups(subset='test' ,categories=['sci.med'])

# Sample a sequence of T characters from the dataset
# that the HMM can read (0=whitespace 1-26=A-Z).
#
# Example of execution:
# O = sample(newsgroups_train.data)
# O = sample(newsgroups_test.data)
#
def sample(data,T=50):
    i = numpy.random.randint(len(data))
    O = data[i].upper().replace('\n',' ')
    O = numpy.array([ord(s) for s in O])
    O = numpy.maximum(O[(O>=65)*(O<90)+(O==32)]-64,0)
    j = numpy.random.randint(len(O)-T)
    return O[j:j+T]

# Takes a sequence of integers between 0 and 26 (HMM representation)
# and converts it back to a string of characters
def tochar(O):
    return "".join(["%s"%chr(o) for o in (O+32*(O==0)+64*(O>0.5))])

### Question 2a (15 P)

In order to train the HMM, we use a stochastic optimization algorithm where the Baum-Welch procedure is applied to randomly drawn sequences of $T=50$ characters at each iteration. The HMM has 27 visible states (A-Z + whitespace) and 200 hidden states. Because the Baum-Welch procedure optimizes for the sequence taken as input, and no necessarily the full text, it can take fairly large steps in the parameter space, which is inadequate for stochastic optimization. We consider instead for the parameters $\lambda = (A,B,\Pi)$ the update rule $\lambda^{new} = (1-\gamma) \lambda + \gamma \bar \lambda$, where $\bar \lambda$ contains the candidate parameters obtained from Equations 40a-c. A reasonable value for $\gamma$ is $0.1$.

* *Create* a new class `HMMChar` that extends the class `HMM` provided in `hmm.py`.
* *Implement* for this class a new method `HMMchar.learn(self)` that overrides the original methods, and implements the proposed update rule instead.
* *Implement* the stochastic training procedure and run it.
* *Monitor* $\log p(O|\lambda)$ on the test set at multiple iterations for sequences of same length as the one used for training. (Hint: for less noisy log-probability estimates, use several sequences or a moving average.)

In [101]:
from hmm import HMM
na = numpy.newaxis

class HMMChar(hmm.HMM):

    def learn(self,param):
        # Compute gamma
        self.gamma = self.alpha*self.beta / self.pobs
        
        # Compute xi and psi
        self.xi = self.alpha[:-1,:,na]*self.A[na,:,:]*self.beta[1:,na,:]*self.Z[1:,na,:] / self.pobs
        self.psi = self.gamma[:,:,na]*(self.O[:,na,na] == numpy.arange(self.B.shape[1])[na,na,:])
        
        # Update HMM parameters
        self.A  = param*(self.xi.sum(axis=0)  / self.gamma[:-1].sum(axis=0)[:,na])+(1-param)*self.A
        self.B  = param*(self.psi.sum(axis=0) / self.gamma.sum(axis=0)[:,na])+(1-param)*self.B
        self.Pi = param*(self.gamma[0])+(1-param)*self.Pi

    def generate(self, T):
        N = numpy.arange(self.Pi.shape[0])
        M = numpy.arange(self.B.shape[1])
        states = numpy.zeros(T)
        symbols = numpy.zeros(T, dtype=int)
        states[0] = numpy.random.choice(N, size=1, p=self.Pi)
        symbols[0] = numpy.random.choice(M, size=1, p=self.B[states[0],:])
        for s in range(1, T):
            states[s] = numpy.random.choice(N, size=1, p=self.A[states[s-1], :])
            symbols[s] = numpy.random.choice(M, size=1, p=self.B[states[s], :])
        return symbols


hmmchar = HMMChar(200, 27)
trainsample = lambda: sample(newsgroups_train.data)
testsample  = lambda: sample(newsgroups_test.data)

def question2a(hmmchar, trainsample, testsample, iterations=100):
    result = numpy.zeros(iterations)
    for k in range(iterations):
        hmmchar.loaddata(trainsample())
        hmmchar.forward()
        hmmchar.backward()
        hmmchar.learn(0.1)
        result[k] = numpy.log(forward(hmmchar.A, hmmchar.B, hmmchar.Pi, testsample()))
    return result

question2a(hmmchar, trainsample, testsample)

array([-162.10062251, -156.580757  , -156.09755696, -151.15650923,
       -150.16632562, -149.522913  , -153.0994435 , -149.69479725,
       -145.26376158, -150.73024066, -148.73867015, -154.77751895,
       -143.37566415, -142.97677509, -144.54481893, -134.68602492,
       -146.90470661, -149.5882919 , -148.95227357, -140.75975492,
       -140.98639013, -148.09081874, -145.8297605 , -145.22869217,
       -148.56791964, -138.20774599, -143.78076937, -140.39063153,
       -131.59056417, -145.08543598, -149.37908783, -141.89485938,
       -111.98991447, -143.07721399, -136.83723701, -148.84195832,
       -141.7326714 , -146.83664151, -141.37673283, -139.92763161,
        -97.83790217, -155.44510494, -137.99978625, -139.76884504,
       -136.4191746 , -149.46230115, -140.16123093, -140.45278159,
       -136.35888852, -150.36278061, -133.59425922, -141.54617982,
       -130.7475347 , -139.25002393, -145.04625481, -138.86762523,
       -159.68170565, -155.09928778, -146.59798147, -140.53927

### Question 2b (15 P)

In order to visualize what the HMM has learned, we would like to generate random text from it. A well-trained HMM should generate character sequences that have some similarity with the text it has been trained on.

* *Implement* a method `generate(self,T)` of the class `HMMChar` that takes as argument the length of the character sequence that has to be generated.
* *Test* your method by generating a sequence of 250 characters and comparing it with original text and a purely random sequence.
* *Discuss* how the generated sequences compare with written English and what are the advantages and limitations of the HMM for this problem.

In [102]:
print("original:\n"+tochar(sample(newsgroups_test.data, T=250)))
print("\nlearned:\n"+tochar(hmmchar.generate(250)))
print("\nrandom:\n" +tochar(numpy.random.choice(27, size=250)))

original:
YS OBVIOUSLY I WOULD REIMBURSE FOR YOU ALL POSTAGE AND RELATED CHARGES FAILING THAT IT WOULD BE BENEFICIAL IF ANYONE COULD POINT TO ANY LIBRARY IN THE NY NJ OR PA AREA THAT MAY HAVE THESE BOOKS  PLEASE RESPOND BY EMAIL SINCE I DO NOT READ THIS NEWSGR

learned:
P BL JTOCHMM ULHEAOAXNR YLDHECE YOSITAOSTRN P M COTENGLSSNT C IRSINSTHY UEHLCTSTOE  WRWO S WRDE TUY WAT DYEN IT LURHSCRE  ENCTEYMM E ORSR ALOCO  OLGDA WH  EWAF SE Y KOENE ES LO ESOTHRDYA AREAI LES W ENP AWII O  ECHEN ECWCHIN ADI AGENDNPMELRH  BRENJAE

random:
FHBCRJRJBANVJCKDIJNWJOHBZGJLBRVYPZONXUCUIEKRVLYSXESOTEQDUYCTKTUNSJVLEABVNZROEVWIGH CDTCYVAZKIDJPKDNQUAP VRX WQWOBIRPHT PGBNVTVUPMZHBZLKBX EZHHLWAFVHSO MQSUTELQLMNRPNNVLCN FCTDFULAZQ YBBKF DOFUPLJSJNGMARQDDIFSPXYMJQWL AGQUWKHJCURZTXLQZKXNKIGYIJJDENFXB


/home/mtambos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:25: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/mtambos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:27: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/mtambos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


**ANSWER:**

Although no real english words were generated, in general the word length seems to be reasonable (although it looks like there's a preference for longer words).

The spaces also look right, since there are only a few cases with more than one space between words.

In contrast, the randomly generated sequence generated many more overly long words.

Maybe one solution for generating passable english senteces could be to use words as symbols instead of characters. However, this would imply several orders of magnitude more symbols.

Another experiment using 10 times as many states and training for 5 times as many iterations (see below) show some plausible words (`ON`, `WAR`, `THE`, `SARGE`, etc.). This suggest that  the HMM is capturing some of the structure in the corpus.

As configured, the HMM does not perform well for the task of generating plausible english words.
One advantage of it is that it's relatively simple to have an idea of how a string was generated (by backtracking)

In [105]:
hmmchar = HMMChar(2000, 27)
trainsample = lambda: sample(newsgroups_train.data)
testsample  = lambda: sample(newsgroups_test.data)

question2a(hmmchar, trainsample, testsample, 500)
"learned:\n", tochar(hmmchar.generate(250))

/home/mtambos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:25: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/mtambos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:27: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/mtambos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


('learned:\n',
 ' LT MOLD SIRATOLDSORBOTERTS SIL TONHI DKO AUSTS PEANY ON ART TOMGARKEW WAR TOMEAME CHE TARANT THE TOLH THABEIMY  SAGROSUTS ARAS THE SO EWEPSTECUUTE ITITHEK COTHONS SORSY PELTOYLS TRER EVE INK TO TA S IN IN O QTEA SARGE SANT THOWO CITAS OF LS SYR EAK ')